# Aim: Find and Download any coincident ICESat-2 tracks and Sentinel-1/2 imagery within a given geographic area and date range. 

## Import Libraries

In [3]:
import pandas as pd                 #DataFrames
import geopandas as gpd             #GeoDataFrames
import icepyx as ipx                #Connect to Icesat-2 API
from sentinelsat import SentinelAPI #Connect to Sentinel API
import os                           #General Utilities 
import shapely.wkt                  #Manipulate geometry
from shapely.geometry import Point  #Points 
from zipfile import ZipFile         #Unzip files
import readers as rd                #Convert H5 to Dataframe
import h5py                         #Read H5 
from datetime import datetime, date, timedelta 

## User Inputs

In [4]:
#IS2 Product type (ATL03:Photon Heights);(ATL06:Land Ice Height);(ATL07:Sea Ice Height);(ATL10:Freeboard)
p_name = 'ATL10'   

#Sentinel-1 or Sentinel-2
platform = 'Sentinel-1'

#Maximum cloud cover (For Sentinel-2 Only)
max_cloud = 30

#Date Range as (YYYY,M,D)
date_1 = date(2020,6,1) 
date_2 = date(2020,10,1)

#Search Footprint for Sentinel images can be any WKT geometry. 
#https://arthur-e.github.io/Wicket/sandbox-gmaps3.html
footprint = 'POINT(-48.78025092888943 66.77289756577241)'

#(Optional) Search footprint for is2 track. Useful when looking at specific areas. If blank will use above footprint instead
is2_footprint = 'POLYGON((-49.28562202263943 68.16996339879951,-48.31882514763943 68.16996339879951,-48.31882514763943 65.28249233286489,-49.28562202263943 65.28249233286489,-49.28562202263943 68.16996339879951))'

#Connect to Copernicus API for Sentinel
global api
api = SentinelAPI('username', 'password', 'https://scihub.copernicus.eu/dhus/')

#Enter user details to connect to Icesat-2 API
earthdata_uid = 'username'
email = 'email'

#Where to output folders
out_path = 'E:/Cryo2ice/pycesat/outputs/'

## Finding Sentinel Images

These Functions use user inputs to retrieve a list of Sentinel-2 Images

In [ ]:
def find_S2_images(footprint, date_1, date_2, api, max_cloud):
    #Get all products from user imputs
    products = api.query(footprint, 
                         date=(date_1, date_2), 
                         platformname = 'Sentinel-2', 
                         cloudcoverpercentage=(0, max_cloud),
                         area_relation='Contains', #Can be changed 
                         producttype='S2MSI1C') 
    
    df = api.to_geodataframe(products)
    os.remove('product_list.csv') #Comment out if needed
    df.to_csv('product_list.csv', sep=',')   
    global pid_list
    pid_list = df.index
    return pid_list


def find_S1_images(footprint, date_1, date_2, api):
    #Get all products from user imputs
    products = api.query(footprint, 
                         date=(date_1, date_2), 
                         platformname = 'Sentinel-1', 
                         sensoroperationalmode = 'EW', #EW is low Res, IW high res
                         producttype='GRD', #GRD gridded geocoded, SLC for 
                         area_relation='Contains')
    
    df = api.to_geodataframe(products) #Comment out if needed
    os.remove('product_list.csv')
    df.to_csv('product_list.csv', sep=',')    
    global pid_list
    pid_list = df.index
    return pid_list

## Finding Coincident ICESat-2 tracks

The following function is run for each Sentinel image in pid_list

In [ ]:
def find_IS_tracks(pid, p_name, df, api, footprint='Sentinel'):
    #If is_footprint was not input by user, search the entire footprint of the Sentinel Image
    if footprint == 'Sentinel':  
        footprint = api.get_product_odata(pid).get('footprint')
        
    polygon = shapely.wkt.loads(footprint)
    polygon = str(polygon.bounds)[1:-1]
    polygon = polygon.split(sep=',')
    polygon = [float(i) for i in polygon]
    
    s1_date = api.get_product_odata(pid, full = True).get('Sensing start')
    s1_name = api.get_product_odata(pid, full = True).get('title')
    
    #Calculate time window around Sentinel Image
    #Note: Icepyx does not currently support time periods shorter than a day, therefore hours=1 does nothing here....
    #... The date_range used is one day. Unfortunately this means some images near Midnight might not be used. 
    min_time = (s1_date - timedelta(hours=1)) 
    max_time = (s1_date + timedelta(hours=1))
    date_range = [str(s1_date - timedelta(hours=1))[:10], str(s1_date + timedelta(hours=1))[:10]]
    
    #Run API Query & store products in region_a
    region_a = ipx.Query(p_name, polygon, date_range)
    
    #Print How many tracks pass through the image on the same day
    print(len(region_a.granules.avail))
    
    #Pretty sure this was for bug testing but I am afraid to remove it. (Check back later)
    global co_track_list
    co_track_list = [] 
    global other_tracks
    other_tracks = []
    
    #For each image Calculate exact time interval between Sentinel image and ICESat-2 track
    for dic in region_a.granules.avail:
        item = dic['time_start']
        is_time = datetime.datetime.strptime(item, '%Y-%m-%dT%H:%M:%S.%fZ') #Extract datetime from string
        is_time_dif = abs(is_time - s1_date)
        if min_time < is_time < max_time:
            co_track_list.append(is_time_dif)
        else:
            other_tracks.append(is_time_dif)
        #Append Coincident tracks and images to the end of the Dataframe
        df.loc[len(df)] = [dic['producer_granule_id'], is_time, s1_name, s1_date, is_time_dif, pid, polygon]

## Downloading the ICESat-2 Tracks

In [ ]:
def dl_is2_data(time_gap, df, p_name, directory_path, earthdata_uid, email):
    #Check that the User wants to download the products
    input('Press Enter to order & download the ICESat-2 products') 
    n = 0
    #for each track in dataframe, download if the time gap is lower than input
    #Note: If the download fails, the Sentinel image will still be downloaded and vice-versa. Delete DF row perhaps?
    for index, row in df.iterrows():
    
        if row['time_dif'] < time_gap:
            n = n + 1b
            
            print('Ordering {}:'.format(row['is_name']))

            date = str(row['is_date'].year) + '-' + str(row['is_date'].month) + '-' + str(row['is_date'].day)
            
            #create directory for output
            if os.path.isdir(directory_path + '/' + date) == False:
                os.mkdir(directory_path + '/' + date)
                
            out_path = directory_path + '/' + date
            print(date)
            print(row['is_date'])
            #Download the track
            try:
                tracks = ipx.Query(p_name, row['polygon'], date_range=[date, date])
                tracks.earthdata_login(earthdata_uid, email)
                tracks.order_granules(email=False)
            except:
                print('protocol error')
            try:
                tracks.download_granules(out_path)
            except:
                print('http error, file not found')

## Downloading the Sentinel-1/2 Images

This works for both Sentinel-1 and Sentinel-2

In [ ]:
def dl_sentinel_data(time_gap, df, directory_path, api):
    #Check that the User wants to download the products
    input('Press Enter to order & download the Sentinel products')
    #for each track in dataframe, download if the time gap is lower than input
    #Note: If the download fails, the Sentinel image will still be downloaded and vice-versa. 
    for index, row in df.iterrows():
        if row['time_dif'] < time_gap:
            date = str(row['is_date'].year) + '-' + str(row['is_date'].month) + '-' + str(row['is_date'].day)
            out_path = directory_path + '/' + date
            print('Ordering {}:'.format(row['s_name']))
            api.download(row['s_id'], directory_path=out_path)

## Converting IS2 H5 to Shapefile

For Future: Plot the Height/Freeboard and save as png during this step

In [ ]:
#For reading ATL10
def getATL10(f, beam):
    print('reading atl10')
    lons = f[beam + '/freeboard_beam_segment/beam_freeboard/longitude'][:]
    lats = f[beam + '/freeboard_beam_segment/beam_freeboard/latitude'][:]
    freeboard = f[beam + '/freeboard_beam_segment/beam_freeboard/beam_fb_height'][:]
    ssh_flag = f[beam + '/freeboard_beam_segment/height_segments/height_segment_ssh_flag'][:]
    stype = f[beam + '/freeboard_beam_segment/height_segments/height_segment_type'][:]
    
    df10 = pd.DataFrame({'lats':lats, 'lons':lons, 'freeboard':freeboard, 'stype':stype, 'ssh_flag':ssh_flag})
    return df10

#Main Function
def h5_to_shp(h5_file, out_path, name, p_name):
    #integrate into download
    
    f = h5py.File(h5_file, 'r')
    #Read columns based on product
    if p_name == 'ATL07':
        #Check which orientation is the strong beam
        if ['orbit_info/sc_orient'][0] == 0:
            df_1 = rd.getATL07(f, 'gt1l')
            df_2 = rd.getATL07(f, 'gt2l')
            df_3 = rd.getATL07(f, 'gt3l')
        else:
            df_1 = rd.getATL07(f, 'gt1r')
            df_2 = rd.getATL07(f, 'gt2r')
            df_3 = rd.getATL07(f, 'gt3r')
    elif p_name == 'ATL06':
        if ['orbit_info/sc_orient'][0] == 0:
            df_1 = rd.getATL06(f, 'gt1l')
            df_2 = rd.getATL06(f, 'gt2l')
            df_3 = rd.getATL06(f, 'gt3l')
        else:
            df_1 = rd.getATL06(f, 'gt1r')
            df_2 = rd.getATL06(f, 'gt2r')
            df_3 = rd.getATL06(f, 'gt3r')
        
    elif p_name == 'ATL03':
        if ['orbit_info/sc_orient'][0] == 0:
            df_1 = rd.getATL03(f, 'gt1l')
            df_2 = rd.getATL03(f, 'gt2l')
            df_3 = rd.getATL03(f, 'gt3l')
        else:
            df_1 = rd.getATL03(f, 'gt1r')
            df_2 = rd.getATL03(f, 'gt2r')
            df_3 = rd.getATL03(f, 'gt3r')
    
    elif p_name == 'ATL10':
        if ['orbit_info/sc_orient'][0] == 0:
            df_1 = getATL10(f, 'gt1l')
            df_2 = getATL10(f, 'gt2l')
            df_3 = getATL10(f, 'gt3l')
        else:
            df_1 = getATL10(f, 'gt1r')
            df_2 = getATL10(f, 'gt2r')
            df_3 = getATL10(f, 'gt3r')
            
    #Get coordinates of each point and convert to Shapely Point         
    geometry_1 = [Point(xy) for xy in zip(df_1.lons,df_1.lats)]
    geometry_2 = [Point(xy) for xy in zip(df_2.lons,df_2.lats)]
    geometry_3 = [Point(xy) for xy in zip(df_3.lons,df_3.lats)]
    
    #Load into GeoDataFrame
    gdf_1 = gpd.GeoDataFrame(df_1, crs="EPSG:4326", geometry=geometry_1)
    gdf_2 = gpd.GeoDataFrame(df_2, crs="EPSG:4326", geometry=geometry_2)
    gdf_3 = gpd.GeoDataFrame(df_3, crs="EPSG:4326", geometry=geometry_3)
    
    
    #Convert to Shapefile
    print('Writing Left beam....')
    gdf_1.to_file(out_path + '/' + name +  '_1.shp')
    print('Writing Centre beam....')
    gdf_2.to_file(out_path + '/' + name + '_2.shp')
    print('Writing Right beam....')
    gdf_3.to_file(out_path + '/' + name + '_3.shp')
    print('Done!')

## Extract tiff from Sentinel-1 Zip

Extracts all Polarizations (Usually HH and HV)
Only works for GRD products (not SLC)

In [ ]:
def extract_S1_tiff(zip_file, destination):
    with ZipFile(zip_file, 'r') as zipObj:
        file_list = zipObj.namelist()
        for filename in file_list:
            if filename.endswith('.tiff') or filename.endswith('.tiff'):
                print('Extracting ' + filename)
                zipObj.extract(filename, destination)

## Combining these functions

In [ ]:
#Get all Sentinel images
if platform = 'Sentinel-1':
    pid_list = si.find_S1_images(footprint, date_1, date_2, api)
elif platform = 'Sentinel-2':
    pid_list = si.find_S1_images(footprint, date_1, date_2, api, max_cloud)
print('%i Images found'%len(pid_list))

#DataFrame to store tracks & images
df = pd.DataFrame(columns=['is_name', 'is_date', 's_name', 's_date', 'time_dif', 's_id', 'polygon'])

#For each Sentinel images, search for ICESat-2 tracks 
for pid in pid_list:
    try:
        si.find_IS_tracks(pid, p_name, df, api, is2_footprint) #footprint optional argument to restrict is_2 to section of sentinel image
        print('Finding tracks for {}'.format(pid) )
    except AssertionError:
        print('No tracks found')

#Sort & Display all coincident tracks found to the User along with their time separation        
df=df.sort_values(by='time_dif', ascending=True)
df=df.drop_duplicates(subset='is_name',keep='first')
for index, row in df.iterrows():
    print(str(row['time_dif']) + ' | ' +  row['is_name'] + ' | ' + row['s_name'])

#Let the user decide how many hours the time window should be
#Note This could be hard coded, but leaving it allows the user to decide how many images to download
input_time = float(input('Enter time window for download in hours:'))
time_gap = timedelta(hours=input_time)

#Get time to create a folder to store output. This prevents the creation of duplicate folders
now_time = datetime.now()
now = now_time.strftime('%d%m%Y_%H%M%S')
os.mkdir(out_path + 'IS2_' + platform + '_'  + now)
dest_folder = out_path + 'IS2_' + platform + '_'  + now

#Create a CSV to keep record of the coicident files
df.to_csv(dest_folder + '/' + 'output.csv', sep=',')

#Download the IS2 data
#Note: At present, the user is prompted to enter the password for each individual download
#There must be a way to avoid this but not sure how. 
si.dl_is2_data(time_gap, df, p_name, dest_folder, earthdata_uid, email)

#Iterate through the output folders and convert them all to shapefiles
for root,dirs,files in os.walk(dest_folder, topdown=False):
    for name in files:
       f_path = os.path.join(root, name)
       if f_path.endswith('.h5'):
           print('Converting ' + name + ' to Shapefile')
           out_path = root + '/georef_tracks'
           if os.path.isdir(out_path) == False:
               os.mkdir(out_path)
           try:
               si.h5_to_shp(f_path, out_path, name, p_name)
           except KeyError:
               print('KeyError')
           
           
#Download the Sentinel data
dl_sentinel_data(time_gap, df, dest_folder, api)


  
 # Finding Sentinel 1/2 Images for given Icesat-2 Tracks
  

In this setup, the user enters a directory full of ICESat-2 hdf5 files (is_path). The script extracts the time and location of the track and searches for nearby Sentinel Images

## Extracting Coordinates from IS-2 track

In [ ]:
def get_is_footprint(track,pname):        
    f = h5py.File(track, 'r')
    
    #Read product
    if pname == 'atl10':
        if f['orbit_info/sc_orient'][0] == 0:
            print('left')
            df_1 = rd.getATL10data(track, 'gt1l')
        else:
            print('right')
            df_1 = rd.getATL10data(track, 'gt1r')         
    
    if pname == 'atl07':
        if f['orbit_info/sc_orient'][0] == 0:
            print('left')
            df_1 = rd.getATL07(f, 'gt1l')            
        else:
            print('right')
            df_1 = rd.getATL07(f, 'gt1r')
    
        
    df_len = df_1.shape[0]
    print(df_len)
    
    #Get Coorodinates of points along track
    #This is not a good solution as it could miss images or find the same image twice, but is the only way I could get ..
    #it working consistently
    
    lat1, lon1 = df_1.lats.iloc[0], df_1.lons.iloc[0]
    lat2, lon2 = df_1.lats.iloc[round(df_len/4)], df_1.lons.iloc[round(df_len/4)]
    lat3, lon3 = df_1.lats.iloc[round(df_len/2)], df_1.lons.iloc[round(df_len/2)]
    lat4, lon4 = df_1.lats.iloc[round(df_len*0.75)], df_1.lons.iloc[round(df_len*0.75)]
    lat5, lon5 = df_1.lats.iloc[-1], df_1.lons.iloc[-1]
    
    footprint_1 = 'POINT('+str(lon1)+' '+str(lat1)+')'
    footprint_2 = 'POINT('+str(lon2)+' '+str(lat2)+')'
    footprint_3 = 'POINT('+str(lon3)+' '+str(lat3)+')'
    footprint_4 = 'POINT('+str(lon4)+' '+str(lat4)+')'
    footprint_5 = 'POINT('+str(lon5)+' '+str(lat5)+')'
    
    footprint_list = [footprint_1, footprint_2, footprint_3, footprint_4, footprint_5]
    print('got here')
    
    return footprint_list  

##  Finding and Downloading Sentinel-1/2

In [ ]:
def find_sentinel1(footprint_list, date_1, date_2, api, out_path):
    global pid_list
    pid_list = []
    for n in footprint_list:
        print(n)
        products = api.query(n, 
                             date=(date_1, date_2), 
                             platformname = 'Sentinel-1', 
                             sensoroperationalmode = 'EW',
                             producttype='GRD',
                             area_relation='Intersects')
        if api.get_products_size(products) != 0:
            print(products)
            api.download_all(products, directory_path=out_path)   
    return pid_list

def find_sentinel2(footprint_list, date_1, date_2, api, out_path, max_cloud):
    
    global pid_list
    pid_list = []  
    for n in footprint_list:
        products = api.query(n, 
                             date=(date_1, date_2), 
                             platformname = 'Sentinel-2', 
                             cloudcoverpercentage=(0, max_cloud),
                             area_relation='Intersects',
                             producttype='S2MSI1C')

        if api.get_products_size(products) != 0:
            print(products)
            api.download_all(products, directory_path=out_path)
    return pid_list

## Combining all three functions

In [ ]:

is_path = 'E:/Cryo2ice/cryo2ice_august/'
api = SentinelAPI('username', 'password', 'https://scihub.copernicus.eu/dhus')
pname = 'atl07' #This should maybe be identified from filename to support different porducts
    
#iterate through all is2 files
for i in os.listdir(is_path):
    if i.endswith('.h5'):
        track = is_path + i
        #extract date
        ac_time = i[9:23]
        is_time = datetime.datetime.strptime(ac_time, '%Y%m%d%H%M%S')
        two_hours = datetime.timedelta(hours=6)#Changes hours to limit sea ice drift
        before_time = is_time - two_hours
        after_time = is_time + two_hours
        footprint_list = get_is_footprint(track,pname) #Get IS2 coordinates
        out_path = is_path + ac_time + '/'
        os.mkdir(out_path)  #Put output into new file and copy track into
        copyfile(track, out_path + i)
        #Find Sentinel1/2 change function to whichever is wanted. If Sentinel-2, max_cloud is needed
        
        find_sentinel2(footprint_list, before_time, after_time, api, out_path, max_cloud=60)
        print('Finding Sentinel images')
        